In [27]:
import tensorflow as tf
from deeplite.tf_profiler.tf_profiler import TFProfiler

In [28]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
try:
    tf.compat.v1.enable_eager_execution()
except Exception:
    pass

In [29]:
from deeplite.tf_profiler.tf_profiler import *
from deeplite.profiler import Device, ComputeEvalMetric
from deeplite.tf_profiler.tf_inference import get_accuracy, get_missclass
import pathlib
import numpy as np

In [30]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
x_train = np.random.randn(800, 32,32,3)
x_test = np.random.randn(200, 32, 32, 3)
y_train = np.random.randint(0,100, size=800)
y_test = np.random.randint(0,100, size=200)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = np.eye(100)[y_train.reshape(-1)]
y_test = np.eye(100)[y_test.reshape(-1)]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
        .shuffle(buffer_size=x_train.shape[0]) \
        .batch(128)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
        .batch(128)
data_splits = {'train': train_dataset, 'test': test_dataset}

In [31]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input#tf.keras.applications.vgg19.preprocess_input
base_model = tf.keras.applications.MobileNetV2(input_shape=(32, 32, 3),
                                            include_top=False,
                                            weights='imagenet')
# base_model = tf.keras.models.load_model('mobv3model.h5')
inputs = tf.keras.Input(shape=(32, 32, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)

In [32]:
outputs = tf.keras.layers.Dense(100)(x)
model = tf.keras.Model(inputs, outputs)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(), metrics=['accuracy'])

In [33]:
data_loader = TFProfiler.enable_forward_pass_data_splits(data_splits)
profiler = TFProfiler(model, data_loader, name="Original Model")
profiler.register_profiler_function(ComputeFlops())
profiler.register_profiler_function(ComputeSize())
profiler.register_profiler_function(ComputeParams())
profiler.register_profiler_function(ComputeLayerwiseSummary())
profiler.register_profiler_function(ComputeExecutionTime())
profiler.register_profiler_function(ComputeEvalMetric(get_accuracy, 'accuracy', unit_name='%'))

In [34]:
res = profiler.compute_network_status(batch_size=1, device=Device.CPU, short_print=False,
                                                include_weights=True, print_mode='debug')

In [35]:
print(res)

{'flops': 0.0085980505, 'model_size': 8.972061157226562, 'memory_footprint': 9.033966064453125, 'total_params': 2.351972, 'layerwise_summary': '_________________________________________________________________\nLayer (type)                 Output Shape              Param #   \n=================================================================\ninput_8 (InputLayer)         [(None, 32, 32, 3)]       0         \n_________________________________________________________________\ntf_op_layer_tf_op_layer_true (None, 32, 32, 3)         0         \n_________________________________________________________________\ntf_op_layer_tf_op_layer_sub_ (None, 32, 32, 3)         0         \n_________________________________________________________________\nmobilenetv2_1.00_224 (Model) (None, 1, 1, 1280)        2257984   \n_________________________________________________________________\nglobal_average_pooling2d_3 ( (None, 1280)              0         \n____________________________________________________

In [17]:
len(data_loader['train'])

7

In [10]:
res

{'flops': 0.0085980505,
 'model_size': 8.972061157226562,
 'memory_footprint': 9.033966064453125,
 'total_params': 2.351972,
 'layerwise_summary': '_________________________________________________________________\nLayer (type)                 Output Shape              Param #   \n=================================================================\ninput_2 (InputLayer)         [(None, 32, 32, 3)]       0         \n_________________________________________________________________\ntf_op_layer_tf_op_layer_true (None, 32, 32, 3)         0         \n_________________________________________________________________\ntf_op_layer_tf_op_layer_sub  (None, 32, 32, 3)         0         \n_________________________________________________________________\nmobilenetv2_1.00_224 (Model) (None, 1, 1, 1280)        2257984   \n_________________________________________________________________\nglobal_average_pooling2d (Gl (None, 1280)              0         \n________________________________________________